In [3]:
from ase.io import write,read
from ase.visualize import view
from src.models.loss.validation import _structural_validity
import torch
from pymatgen.io.ase import AseAtomsAdaptor

import json
import sys
import warnings
from pathlib import Path
from typing import Dict, Tuple
from multiprocessing import cpu_count, get_context
import argparse
import contextlib
import io

from ase import Atoms
from ase.io import read
from ase.optimize import LBFGS
from fairchem.core import pretrained_mlip, FAIRChemCalculator
from tqdm import tqdm
from ase.constraints import FixAtoms

# Suppress warnings
warnings.filterwarnings('ignore')

OC20_GAS_PHASE_ENERGIES = {
    'H': -3.48483361833793,
    'O': -7.185616160375758,
    'C': -7.232295041080779,
    'N': -8.09079187764214,
} # Computed by UMA
# Reference molecules : N2, H2O, H2, CO
# E(C) = E(CO) - E(H2O) + E(H2)
# E(H) = 0.5 * E(H2)
# E(N) = 0.5 * E(N2)
# E(O) = E(H2O) - E(H2)

def get_adsorbate_energy_from_table(atoms_obj):
    """Calculate adsorbate energy from lookup table."""
    total_energy = 0.0
    symbols = atoms_obj.get_chemical_symbols()
    
    for atom_symbol in symbols:
        try:
            total_energy += OC20_GAS_PHASE_ENERGIES[atom_symbol]
        except KeyError:
            raise ValueError(
                f"Energy table does not contain '{atom_symbol}' atom. "
                f"Currently supported atoms are {list(OC20_GAS_PHASE_ENERGIES.keys())}."
            )
            
    return total_energy


def get_uma_calculator(model_name: str = "uma-s-1p1", device: str = "cuda") -> FAIRChemCalculator:
    """Get UMA calculator."""
    predictor = pretrained_mlip.get_predict_unit(model_name, device=device)
    return FAIRChemCalculator(predictor, task_name="oc20")


def relaxation_and_compute_adsorption_energy(
    calc: FAIRChemCalculator, 
    system: Atoms, 
    slab: Atoms, 
    adsorbate: Atoms
) -> Tuple[float, float, float, float, bool, bool]:
    try:
        system.calc = calc
        slab.calc = calc
        
        opt = LBFGS(system, logfile=None)
        with contextlib.redirect_stdout(io.StringIO()):
            converged_system = opt.run(0.05)
        steps_system = opt.get_number_of_steps()
        
        opt = LBFGS(slab, logfile=None)
        with contextlib.redirect_stdout(io.StringIO()):
            converged_slab = opt.run(0.05)
        steps_slab = opt.get_number_of_steps()
        
        e_sys = system.get_potential_energy()
        e_slab = slab.get_potential_energy()
        e_adsorbate = get_adsorbate_energy_from_table(adsorbate)
        e_ads = e_sys - (e_slab + e_adsorbate)
        
        print(f"System converged in {steps_system} steps.")
        print(f"Slab converged in {steps_slab} steps.")

        return e_ads, e_sys, e_slab, e_adsorbate, converged_system, converged_slab

    except Exception as e:
        print(f"WARNING: UMA calculation failed for a sample. Error: {e}", file=sys.stderr)
        e_adsorbate = get_adsorbate_energy_from_table(adsorbate)
        return 999.0, float('nan'), float('nan'), e_adsorbate, False, False

adaptor = AseAtomsAdaptor()

idx=0
struct_path = f"/home/jovyan/MinCatFlow/unrelaxed_samples/de_novo_generation/{idx}.traj"
gen_system = read(struct_path)

gen_system.center()

# c = FixAtoms(indices=[atom.index for atom in gen_system if atom.tag == 0])
# gen_system.set_constraint(c)

# calc = get_uma_calculator()
# slab = gen_system.copy()[gen_system.get_tags() != 2]
# adsorbate = gen_system.copy()[gen_system.get_tags() == 2]
# e_ads, e_sys, e_slab, e_adsorbate, converged_system, converged_slab = relaxation_and_compute_adsorption_energy(
#     calc, 
#     gen_system, 
#     slab, 
#     adsorbate
# )
# print("E_ads:", e_ads)

view(gen_system, viewer='ngl')